# Curso Teste A/B/n - PA James

## 0.1. Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

from statsmodels.stats.gof import chisquare_effectsize
from statsmodels.stats.power import GofChisquarePower, TTestIndPower
from statsmodels.sandbox.stats.multicomp import multipletests
from itertools import combinations

# 1.0. Carregar Dados

In [2]:
d = { 'variant': ['interact', 'connect', 'learn', 'help', 'services'],
      'visits': [10283, 2842, 2747, 3180, 2064],
      'clicks_all': [3714, 1587, 1652, 1717, 1348],
      'clicks_link': [42, 53, 21, 38, 45]}

data = pd.DataFrame(d)
data['conversion'] = data['clicks_link'] / data['clicks_all']
data

,variant,visits,clicks_all,clicks_link,conversion
0,interact,10283,3714,42,0.011309
1,connect,2842,1587,53,0.033396
2,learn,2747,1652,21,0.012712
3,help,3180,1717,38,0.022132
4,services,2064,1348,45,0.033383


# 2.0. Design de Experimentos

## 2.1. Formulação das Hipóteses

In [3]:
# H0: Não á nenhuma diferença entre o CTR das variantes da página
# H1: Há uma diferença entre os CTR das variantes da página
#
# Teste Bi-Caudal

## 2.2. Parâmetros do Experimento

In [4]:
k = len(data['clicks_all'])
actual_dist = data['clicks_link'] / data['clicks_link'].sum()
expected_dist = [1/k]*k

In [5]:
efect_size = chisquare_effectsize(expected_dist, actual_dist)
alpha = 0.05
power = 0.80

## Goodness of fitness
# sample_size = GofChisquarePower().solve_power(
#     effect_size=efect_size,
#     alpha=alpha,
#     power=power,
#     n_bins=(2-1)*(5-1) # (nFeature-1)*(nRows-1)
# )

# Teste de independência (Homogeneidade)
sample_size = TTestIndPower().solve_power(
    effect_size=efect_size,
    alpha=alpha,
    power=power
)

sample_size = np.ceil(sample_size).astype(int)

print(f'Tamanho mínimo da amostra por variante: {sample_size}')
print(f'Tamanho total amostras: {k*sample_size}')

Tamanho mínimo da amostra por variante: 222
Tamanho total amostras: 1110


# 3.0. Conversão da página em Faturamento

In [6]:
data['no_clicks_link'] = data['clicks_all'] - data['clicks_link']
df = data[['variant', 'clicks_link', 'no_clicks_link']]
df.set_index('variant', inplace=True)
df

,clicks_link,no_clicks_link
variant,,
interact,42,3672
connect,53,1534
learn,21,1631
help,38,1679
services,45,1303


In [7]:
chi2, p_value, dof, ex = stats.chi2_contingency(df)
print('Chi Squared: {} - p-value: {}'.format(chi2, p_value))

Chi Squared: 46.33660181942126 - p-value: 2.0959498129984563e-09


# 4.0. Post-hoc Testing

In [8]:
all_combinations = list(combinations(df.index, 2))
p_values = []

for comb in all_combinations:
    new_df = df[(df.index == comb[0]) | (df.index == comb[1])]
    chi2, p_value, dof, ex = stats.chi2_contingency(new_df)
    p_values.append(p_value)

# Correção de Benferroni
reject_list, correct_p_values = multipletests(p_values, method='bonferroni')[:2]

In [9]:
for comb, p_val, corr_p_val, reject in zip(all_combinations, p_values, correct_p_values, reject_list):
    print('\n{}: p_value:{}; corrected_p_value:{}; Reject:{}'.format(comb, p_val, corr_p_val, reject))


('interact', 'connect'): p_value:5.3676772349808135e-08; corrected_p_value:5.367677234980813e-07; Reject:True

('interact', 'learn'): p_value:0.7616980743361713; corrected_p_value:1.0; Reject:False

('interact', 'help'): p_value:0.0031030587017400212; corrected_p_value:0.03103058701740021; Reject:True

('interact', 'services'): p_value:1.798089447385411e-07; corrected_p_value:1.7980894473854111e-06; Reject:True

('connect', 'learn'): p_value:0.00013292868361715983; corrected_p_value:0.0013292868361715984; Reject:True

('connect', 'help'): p_value:0.06144184057612575; corrected_p_value:0.6144184057612575; Reject:False

('connect', 'services'): p_value:1.0; corrected_p_value:1.0; Reject:False

('learn', 'help'): p_value:0.0508958228881819; corrected_p_value:0.5089582288818191; Reject:False

('learn', 'services'): p_value:0.00020374035733741825; corrected_p_value:0.0020374035733741825; Reject:True

('help', 'services'): p_value:0.07301998638337415; corrected_p_value:0.7301998638337415; R

**Resultado do teste:**

    Interact x Connect -> Há uma relação, diferença, dependência -> p_value = 5.367677234980813e-07
    Interact x Services -> Há uma relação, diferença, dependência -> p_value = 1.7980894473854111e-06
    Interact x Help -> Há uma relação, diferença, dependência -> p_value = 0.6144184057612575

    Connect x help -> Não há diferença
    Connect x Services -> Não há diferença
    Help x services -> Não há diferença

